# Функции transform_inn_col(), check_inn()

### Импорт модулей

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField
import pandas as pd

In [2]:
from pysp_lib.check import *

### Инициализация Spark сессии

In [3]:
spark = SparkSession.builder.appName("Test").master("local[*]").getOrCreate()

### Создаем пробный датасет

In [4]:
data_1 = {'company':['OOO Руслан', 'ООО Агроторг', 'ООО Ригла', 'ПАО Сургутнефтегаз', 'ООО Альбион-2002', 'ПАО Щит'
                   , 'АО Эр-Телеком Холдинг', 'OOO Ромашка', 'АО Магистраль', 'ИП Макеев Сергей Николаевич'
                   , 'ИП Савкин Николай Степанович', 'ИП Анчина Валентина Николаевна', 'ИП Михайлова Нина Викторовна'
                   , 'ИП Веремеенко Сергей Евгеньевич', 'ИП Журавлев Анатолий Николаевич', 'ИП Макаров Петр Владимирович'
                  , 'ООО Пальма', 'АО Завод', 'ПАО Кот', 'ООО Веста', 'ООО Дом', 'ЗАО Марс'],
       'inn': ['4707005850', '7825706086', '', '7724211288', '' , '8602060555', '5257056036', '5902202276'
        , '131900231406', '131400060294', '132300043547', '131400027307', '132200001744', '123456790A', '0000000000', '000000000000'
       , '0000000000', '00000000000', 'ASDFTV', '0001234567', '0105020396', '105020396']}

In [5]:
data_2 = {'company': ['ООО Пират', 'АО Музыка', 'ПАО Пластинка', 'ООО Луна', 'АО Сон']
         ,'inn': ['4707005850', '7825706086', '', '105047905', '0101010083']}

In [6]:
pddf_1 = pd.DataFrame(data_1)
pddf_2 = pd.DataFrame(data_2)

In [7]:
pydf_1 = spark.createDataFrame(pddf_1)
pydf_2 = spark.createDataFrame(pddf_2)\
                .withColumnRenamed('inn', 'inn_n')\
                .withColumnRenamed('company', 'company_n')

In [8]:
pydf_union = pydf_1.join(pydf_2, pydf_1.inn == pydf_2.inn_n, 'outer').select('company', 'inn')

Общее количество записей

In [9]:
pydf_union.count()

24

Количество нулевых значений

In [10]:
pydf_union.filter(f.col('inn').isNull()).count()

2

### Вызов функции transform_inn_col()

In [11]:
test_transform_func = transform_inn_col(pydf_union, 'inn')

Количество значений (нулевые значения отфильтрованы)

In [12]:
test_transform_func.count()

22

Смотрим изменившиеся колонки

In [13]:
test_transform_func.select('inn_fixed', 'company')\
                    .exceptAll(test_transform_func.select('inn', 'company'))\
                    .show()

+------------+--------------------+
|   inn_fixed|             company|
+------------+--------------------+
|  0105020396|            ЗАО Марс|
|  0123456790|ИП Веремеенко Сер...|
|            |             ПАО Кот|
|000000000000|            АО Завод|
+------------+--------------------+



### Вызов функции check_inn()

с неотредактированной колонкой ИНН

In [14]:
test_check_inn_func_non_fixed_inn = check_inn(test_transform_func, 'inn')

считаем количество валидных ИНН

In [15]:
test_check_inn_func_non_fixed_inn.groupBy('inn_valid_flag').count().show()

+--------------+-----+
|inn_valid_flag|count|
+--------------+-----+
|             1|   12|
|             0|   10|
+--------------+-----+



с отредактированной колонкой ИНН

In [16]:
test_check_inn_func_fixed_inn = check_inn(test_transform_func, 'inn_fixed')

считаем количество валидных ИНН (благодаря предварительному редактированиб удалось увеличить число валидных ИНН)

In [17]:
test_check_inn_func_fixed_inn.groupBy('inn_fixed_valid_flag').count().show()

+--------------------+-----+
|inn_fixed_valid_flag|count|
+--------------------+-----+
|                   1|   13|
|                   0|    9|
+--------------------+-----+



In [18]:
spark.stop()